# PyCap runs for Little Plover River - based on Wisconsin DNR strategy

This notebook allows you to update pumping rates in the input Excel file, make a run, and return all the metrics. This script will convert that `.xlsx` file into a `.yml`. 


In [ ]:
import yaml
import pandas as pd
import numpy as np
from pathlib import Path
import sys
sys.path.append('../scripts')
from wdnr_pycap import Excel2YML, pycap_metrics
import matplotlib.pyplot as plt
import os, platform
from pycap.analysis_project import Project
import geopandas as gpd
from shapely.geometry import Point
from  datetime import date


In [ ]:
pycap_run_path = Path("../pycap_runs")

background_run_path = pycap_run_path / 'pycap_base'

### Make a map showing all the well locations and pumping rates - you can use this to guide your scenario creation

In [ ]:
GPM2CFS = 1/7.48052/60
with open(background_run_path / "LPR_redux.yml", "r") as ifp:
    indat = yaml.safe_load(ifp)
indcol = [i for i in indat.keys() if 'well' in i.lower()]
names = [indat[i]['name'] for i in indcol]
locs = [Point(-indat[i]['loc']['x'],indat[i]['loc']['y']) for i in indcol]
Q = [indat[i]['Q'] for i in indcol]

wells_gdf = gpd.GeoDataFrame(index = indcol, data={"name":names, 
                                                   "Q_gpm":Q, 
                                                   "Q_cfs": [Qi*GPM2CFS for Qi in Q],
                                                   "geometry":locs}, crs=4326)
wells_gdf.explore(                  
                vmin=0,vmax=1,
                  style_kwds={"style_function":
                                  lambda x: 
                                  {"radius":x["properties"]["Q_gpm"]/20}})

### In the Excel file noted below, you can change pumping rates in the `HCW_Inputs` sheet by changing the `Q_gpm` values (this column should be bold and highlighted). Once you update this and save, you can run through this notebook to get results for all the metrics.

### If you lose track of the original pumping values, you can copy the entire column back from `../Inputs/LPR_Prepped_IDW.xlsx`

In [ ]:
#### Path to excel input file. You can change the name if you want to keep multiple versions of it.
pycap_inputs_excel = Path("../Student_Input/LPR_Single_Run.xlsx")




### NOTE! You should change the `pycap_run_name` variable below to keep track of runs you make. Results will be saved with that name

In [ ]:
pycap_run_name = 'tryanother'

base_run_path = pycap_run_path / 'student_run'

In [ ]:
if not base_run_path.exists():
    base_run_path.mkdir(parents=True)

### Create YML from Excel
##### Created with a function

In [ ]:
project_dict, stream_dict, well_dict, ddresp_dict, combdict = Excel2YML(pycap_inputs_excel,pycap_run_name, base_run_path)
     

### Now run the base model with `pycap`

In [ ]:
yml_filename = f"{pycap_run_name}.yml"

#### We create a `Project`, then report the responses and finally write the responses to an external CSV file

In [ ]:
ap = Project(base_run_path /  yml_filename)
ap.report_responses()
ap.write_responses_csv()

#### Now let's have a look at the main results files
##### First the summary base depletion

In [ ]:
depl = pd.read_csv(
    base_run_path / f"output/{pycap_run_name}.table_report.base_stream_depletion.csv", index_col = 0
)
depl

In [ ]:
all_time_series = pd.read_csv(
    base_run_path / f"output/{pycap_run_name}.table_report.all_ts.csv".format(pycap_run_name), index_col=0
)

cum_depl = all_time_series.sum(axis=1)

fig,ax = plt.subplots(nrows=1, ncols=1)
ax.plot(all_time_series, alpha=0.6, c='k', lw=0.2)
ax.plot(cum_depl, c='r')
ax.set_title('Cumulative Depl, LPR at Hoover,Base Case Analysis')
ax.set_xlabel("Days")
ax.set_ylabel('Depletion, cfs')

In [ ]:
LPR_baseline_flow = 8.6

In [ ]:
all_time_series= LPR_baseline_flow-all_time_series
cum_depl = LPR_baseline_flow - cum_depl

In [ ]:
fig,ax = plt.subplots(nrows=1, ncols=1)
ax.plot(cum_depl, c='r')
ax.set_title('Streamflow, LPR at Hoover,Base Case Analysis')
ax.set_xlabel("Days")
ax.set_ylabel('Streamflow, cfs')

### now what about the metrics?

In [ ]:
all_metrics = pycap_metrics(depl, well_dict,combdict, base_run_path, pycap_run_name)

# FINALLY, let's see what we got!

In [ ]:

all_metrics

In [ ]:
all_metrics.to_csv(base_run_path / f"{pycap_run_name}_results.csv")